In [ ]:
import pandas as pd
import numpy as np
import warnings

In [ ]:
warnings.filterwarnings('ignore')

### Reading the movie dataset

In [ ]:
df = pd.read_csv("ml-100k/u.data",sep='\t',names = ["user_id","item_id","rating","timestamp"])

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df['user_id'].nunique()

In [ ]:
df['item_id'].nunique()

In [ ]:
movie_titles = pd.read_csv("ml-100k/u.item",sep="\|",header = None,encoding='latin-1')

In [ ]:
movie_titles 

In [ ]:
movie_titles = movie_titles[[0,1]]
movie_titles.columns = ['item_id','title']

In [ ]:
movie_titles

In [ ]:
df = pd.merge(df,movie_titles,on = "item_id")
df

### Exploratory Data Analysis

In [ ]:
import matplotlib.pyplot as plt
#!pip install seaborn
import seaborn as sns

In [ ]:
ratings = pd.DataFrame(df.groupby('title').mean()['rating'])
ratings

In [ ]:

ratings['num of ratings'] = pd.DataFrame(df.groupby('title').count()['rating'])

In [ ]:
ratings.sort_values(by=['rating'],ascending = False)

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(ratings['num of ratings'],bins=70)
plt.show()

In [ ]:
plt.hist(ratings['rating'],bins=70)
plt.show()

In [ ]:
sns.jointplot(x='rating',y='num of ratings',data=ratings,alpha = 0.5)

### Recommender

In [ ]:
df

In [ ]:
movie_matrix = df.pivot_table(index = "user_id",columns = "title",values = "rating")

In [ ]:
movie_matrix

In [ ]:
starwars_user_ratings = movie_matrix['Star Wars (1977)']

In [ ]:
starwars_user_ratings

In [ ]:
similar_to_starwars = movie_matrix.corrwith(starwars_user_ratings)

# Corelation = Covariance(x,y) / σ(x)*σ(y)

# Covariance(x,y) = Σ(xᵢ - x̄)*(yᵢ - ȳ) / Total records

# Where,

# σ(y) = Standard Deviation of y

# σ(x) = Standard Deviation of x
# xᵢ = iᵗʰ element of x
# x̄ = sample mean of x
# yᵢ = iᵗʰ element of y
# ȳ = sample mean of y

In [ ]:
similar_to_starwars

In [ ]:
corr_starwars = pd.DataFrame(similar_to_starwars,columns = ['Correlation'])

In [ ]:
corr_starwars.dropna(inplace=True)

In [ ]:
corr_starwars

In [ ]:
corr_starwars.sort_values(by=['Correlation'],ascending = False)

In [ ]:
corr_starwars=corr_starwars.join(ratings['num of ratings'])

In [ ]:
corr_starwars[corr_starwars['num of ratings']>=100].sort_values(by=['Correlation'],ascending = False)

### Recommendation Funtion

In [ ]:
def recommend_movie(movie_name):
    movie_user_ratings = movie_matrix[movie_name]
    similar_to_movie = movie_matrix.corrwith(movie_user_ratings)
    
    corr_movie = pd.DataFrame(similar_to_movie,columns = ['Correlation'])
    corr_movie.dropna(inplace=True)
    
    corr_movie=corr_movie.join(ratings['num of ratings'])
    predictions = corr_movie[corr_movie['num of ratings']>=100].sort_values(by=['Correlation'],ascending = False)
    
    return predictions

In [ ]:
recommend_movie("Titanic (1997)").head(5)

In [ ]:
import pickle

In [ ]:
pickle.dump(df,open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(movie_matrix,open('movie_matrix_dict.pkl','wb'))

 User 1:              User 2:
 Movie 1   3.0        Movie 1   5.0
 Movie 2   2.0        Movie 2   4.0  
 Movie 3   4.0        
 Movie 4   1.0

 Now, we want movie recommendations based on Movie 2
 So, we will be recommended Movie 1 and not Movie 3 or Movie 4 as our choice matches('Corelates') with User 2